In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [2]:
#LIST_PATH
json_path = '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/relevance_data.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [3]:
number_of_study = len(dataset)
print("number of study : ", number_of_study)

number of study :  499


In [40]:
#remove remove_bi_file if it exists in every serie 
from library_dicom.dicom_processor.tools.folders import *
directory_validated_dicom = ''
paths = get_series_path(directory_validated_dicom)

for path in paths : 
    remove_bi_file(path)

In [ ]:
#Check if there is Unconstant Spacing error in every serie of the dataset 
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


In [ ]:
#Careful : maybe need to rewrite path if the series are in an other folder

In [ ]:
#Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV' #if it exist

#save serie_path with false mask 
serie_false_mask = []
#save result about serie with false mask 
results_false_mask = []
#save path of MIP to generate PDF 
path_mip = []

#save error serie 
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))
        subliste = []
        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[-1]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True :  
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")


            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask with threshold in csv file  
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                #create mip for false mask and check 
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin = 0, vmax = 10) 
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys', borne_max = 5.0)
                subliste.append(angle_filename_mask)
                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True : 
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")

            else : 
                print("FALSE MASK")
                false_mask_dataset_1.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask with threshold in csv 
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                #create mip for false mask and check 
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin = 0, vmax = 10) 
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys', borne_max = 5.0)
                subliste.append(angle_filename_mask)
                path_mip.append(subliste)
               

    except Exception as err : 
        print(serie)
        print(err)
        error_dataset.append(serie)


In [ ]:
print("number of error : ", len(error_dataset))
print("number of false mask : " , len(false_mask_dataset))

In [46]:
#Save different json 
directory_nifti = ''
write_json_file(directory_nifti, 'false_mask_serie', serie_false_mask)
write_json_file(directory_nifti, 'false_mask_results', results_false_mask)

write_json_file(directory_nifti, 'error', error_dataset)

mip directory = directory_nifti+'/'+'MIP'
write_json_file(mip_directory, 'path_mip', path_mip)

In [47]:
#create pdf of MIP for false mask 
filename = os.path.join(mip_directory, 'list_mip_dataset.pdf')
create_pdf_mip(path_mip, filename)

In [ ]:
#Generate PET, CT, MASK NIFTI without checking

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV'

error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
          
          
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
           
    except Exception as err : 
        print(err)
        print(serie)
        error_dataset.append(serie)

In [4]:
dataset[0]

['/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/12011101021108/PET_Screening/1.3.46.670589.28.2.10.4.9167.27921.2.1340.0.1363265573/1.3.46.670589.28.2.10.4.9167.27921.2.1340.0.1363265573',
 'PT',
 '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/12011101021108/PET_Screening/1.2.840.113704.1.111.4748.1363263038.8/1.2.840.113704.1.111.4748.1363263038.8',
 'CT',
 '12011101021108',
 '1.2.250.1.59.470.940100027.1.20130308163045.16363.12867',
 'PET_Screening']

In [8]:
#Generate PET & CT NIFTI only

nifti_directory = '/media/deeplearning/Elements/RELEVANCE_NIFTI'

error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if 'PET_Screening' in serie : 
                if serie[1] == 'PT' : 
                        serie_pt_objet = SeriesPT(serie[0]) 
                        nifti_array = serie_pt_objet.get_numpy_array()
                        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                        filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                        serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                        print("EXPORT NIFTI PT")
                                #generation nifti CT
                        serie_ct_objet = SeriesCT(serie[2])
                        serie_ct_objet.get_numpy_array()
                        filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                        serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                        print("EXPORT NIFTI CT")

                else : 
                        serie_pt_objet = SeriesPT(serie[2]) 
                        nifti_array = serie_pt_objet.get_numpy_array()
                        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                        
                                #generation nifti PT
                        filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                        serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                        print("EXPORT NIFTI PT")
                                #generation nifti CT
                        serie_ct_objet = SeriesCT(serie[0])
                        serie_ct_objet.get_numpy_array()
                        filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                        serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                        print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)

ng/1.3.12.2.1107.5.1.4.11063.30000013092407480937500014459/1.3.12.2.1107.5.1.4.11063.30000013092407480937500014459', 'PT', '52011101341109', '1.2.276.0.7230010.3.8666689', 'PET_Screening']
The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. GDCM)
45
46
47
['/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/52011101341113/PET_Screening/1.3.12.2.1107.5.1.4.11063.30000014051308081807800016235/1.3.12.2.1107.5.1.4.11063.30000014051308081807800016235', 'PT', '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/52011101341113/PET_Screening/1.3.12.2.1107.5.1.4.11063.30000014051308063328100008743/1.3.12.2.1107.5.1.4.11063.30000014051308063328100008743', 'CT', '52011101341113', '1.2.276.0.7230010.3.8699476', 'PET_Screening']
The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. GDCM)
48
EXPORT NIFTI PT
EXPORT NIFTI CT
49
50
EXPORT NIFTI PT
EXPORT NIFTI CT
51
52
